In [1]:
from openfisca_uk import Microsimulation
from openfisca_uk import *
from openfisca_core.model_api import Reform
from openfisca_uk.entities import Person, BenUnit, Household

libyaml is not installed in your environment. This can make OpenFisca slower to start. Once you have installed libyaml, run 'pip uninstall pyyaml && pip install pyyaml --no-cache-dir' so that it is used in your Python environment.



In [2]:
sim = Microsimulation()

class PA_reform(Reform):
    def apply(self):
        self.neutralize_variable("personal_allowance")
        
sim_no_PA = Microsimulation(PA_reform)

In [3]:
revenue = sim.calc("net_income").sum()
revenue_diff = revenue - sim_no_PA.calc("net_income").sum()
revenue_diff

96342127753.30627

In [4]:
BI_amount = revenue_diff/(sim.calc("people").sum())
BI_amount

1471.3539074481146

In [5]:
from openfisca_core.model_api import *
from openfisca_uk.tools.general import *

class BI(Variable):
    value_type = float
    entity = Person
    label = u"UBI"
    definition_period = YEAR
    def formula(person, period, parameters):
        return(BI_amount)

class gross_income(Variable):
    value_type = float
    entity = Person
    label = u"Gross income, including benefits"
    definition_period = YEAR

    def formula(person, period, parameters):
        COMPONENTS = [
            "employment_income",
            "pension_income",
            "self_employment_income",
            "property_income",
            "savings_interest_income",
            "dividend_income",
            "miscellaneous_income",
            "benefits",
            "BI"
        ]
        return add(person, period, COMPONENTS)   

class basic_income(Reform):
    def apply(self):
        self.add_variable(BI)
        self.update_variable(gross_income)
        
sim_BI = Microsimulation(PA_reform, basic_income)

In [6]:
sim_BI.calc("household_net_income").mean() - sim.calc("household_net_income").mean()
#household_net_income clips negative values to zero, so changes for people with negative disposable income don't show up
#they go from 0 to 0 rather than e.g. -100 to -50

-6.817517520670663

In [10]:
sim_BI.calc("net_income").mean() - sim.calc("net_income").mean()

-0.0013760499059571885